# Flower Framework Bug Prediction Experiment

## Study: FL Framework Comparison - TFF vs Flower

This notebook evaluates Flower for bug prediction with:
- **Communication Cost**: Tracking bytes sent/received per round
- **Security Analysis**: Model poisoning (gradient +100) and Data poisoning (label flipping)
- **Metrics**: F1 Score, Accuracy, Precision, Recall, Loss (before/after attacks)

## 1. Environment Setup

In [ ]:
# Install required packages
!pip install flwr>=1.5.0 torch>=2.0.0 scikit-learn -q
print("Installation complete!")

In [ ]:
# Import libraries
import os
import json
import time
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import flwr as fl
from flwr.common import NDArrays, Scalar
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Any, Optional
from dataclasses import dataclass, field, asdict
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"Flower version: {fl.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Upload dataset
from google.colab import files

print("Please upload dataset_pairs_1_.json")
uploaded = files.upload()
dataset_path = list(uploaded.keys())[0]
print(f"\nUploaded: {dataset_path}")

## 2. Data Processing

In [ ]:
class CodeTokenizer:
    """Simple tokenizer for Python code."""
    
    def __init__(self, max_vocab_size=5000, max_length=200):
        self.max_vocab_size = max_vocab_size
        self.max_length = max_length
        self.vocab = {'<PAD>': 0, '<UNK>': 1, '<START>': 2, '<END>': 3}
        self.word_counts = {}
    
    def fit(self, codes: List[str]):
        for code in codes:
            tokens = self._tokenize(code)
            for token in tokens:
                self.word_counts[token] = self.word_counts.get(token, 0) + 1
        
        sorted_words = sorted(self.word_counts.items(), key=lambda x: -x[1])
        for word, _ in sorted_words[:self.max_vocab_size - len(self.vocab)]:
            if word not in self.vocab:
                self.vocab[word] = len(self.vocab)
    
    def _tokenize(self, code: str) -> List[str]:
        import re
        tokens = re.findall(r'[a-zA-Z_][a-zA-Z0-9_]*|[0-9]+|[^\s\w]', code)
        return tokens
    
    def encode(self, code: str) -> np.ndarray:
        tokens = self._tokenize(code)
        ids = [self.vocab.get(t, 1) for t in tokens[:self.max_length]]
        padded = ids + [0] * (self.max_length - len(ids))
        return np.array(padded[:self.max_length], dtype=np.int64)


class CodeFeatureExtractor:
    """Extract handcrafted features from code."""
    
    def extract(self, code: str) -> np.ndarray:
        features = [
            len(code), code.count('\n'), code.count('def '),
            code.count('class '), code.count('if '), code.count('for '),
            code.count('while '), code.count('try:'), code.count('except'),
            code.count('return'), code.count('import'), code.count('='),
            code.count('+'), code.count('-'), code.count('*'),
            code.count('/'), code.count('['), code.count('('),
            len(code.split()), code.count('#')
        ]
        return np.array(features, dtype=np.float32)


class BugPredictionDataProcessor:
    """Process bug prediction dataset."""
    
    def __init__(self, tokenizer: CodeTokenizer, feature_extractor: CodeFeatureExtractor):
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
    
    def load_data(self, json_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        all_codes = []
        for pair in data:
            all_codes.append(pair['buggy_code'])
            all_codes.append(pair['fixed_code'])
        
        self.tokenizer.fit(all_codes)
        
        tokens_list, features_list, labels_list = [], [], []
        
        for pair in data:
            # Buggy code (label=1)
            tokens_list.append(self.tokenizer.encode(pair['buggy_code']))
            features_list.append(self.feature_extractor.extract(pair['buggy_code']))
            labels_list.append(1)
            
            # Fixed code (label=0)
            tokens_list.append(self.tokenizer.encode(pair['fixed_code']))
            features_list.append(self.feature_extractor.extract(pair['fixed_code']))
            labels_list.append(0)
        
        return np.array(tokens_list), np.array(features_list), np.array(labels_list)
    
    def partition_iid(self, tokens, features, labels, num_clients):
        indices = np.random.permutation(len(labels))
        splits = np.array_split(indices, num_clients)
        
        client_data = []
        for split in splits:
            client_data.append({
                'tokens': tokens[split],
                'features': features[split],
                'labels': labels[split]
            })
        return client_data

In [ ]:
# Load and process data
tokenizer = CodeTokenizer(max_vocab_size=5000, max_length=200)
feature_extractor = CodeFeatureExtractor()
processor = BugPredictionDataProcessor(tokenizer, feature_extractor)

tokens, features, labels = processor.load_data(dataset_path)
print(f"Total samples: {len(labels)}")
print(f"Buggy (1): {sum(labels)}, Non-buggy (0): {len(labels) - sum(labels)}")
print(f"Token shape: {tokens.shape}, Features shape: {features.shape}")

## 3. PyTorch Model Definition

In [ ]:
class BugPredictionModel(nn.Module):
    """PyTorch model for bug prediction."""
    
    def __init__(self, vocab_size=5000, embedding_dim=64, hidden_dim=128, num_features=20):
        super().__init__()
        
        # Token processing
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        
        # Feature processing
        self.feature_fc = nn.Linear(num_features, 32)
        
        # Combined layers
        self.fc1 = nn.Linear(hidden_dim * 2 + 32, 64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, tokens, features):
        # Token branch
        embedded = self.embedding(tokens)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out[:, -1, :]  # Take last output
        
        # Feature branch
        feat_out = F.relu(self.feature_fc(features))
        
        # Combine
        combined = torch.cat([lstm_out, feat_out], dim=1)
        x = F.relu(self.fc1(combined))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        
        return x.squeeze()


def get_model_parameters(model: nn.Module) -> List[np.ndarray]:
    """Extract model parameters as numpy arrays."""
    return [param.cpu().detach().numpy() for param in model.parameters()]


def set_model_parameters(model: nn.Module, parameters: List[np.ndarray]):
    """Set model parameters from numpy arrays."""
    for param, new_param in zip(model.parameters(), parameters):
        param.data = torch.tensor(new_param, dtype=param.dtype, device=param.device)


def calculate_model_size(model: nn.Module) -> int:
    """Calculate model size in bytes."""
    total_bytes = 0
    for param in model.parameters():
        total_bytes += param.numel() * param.element_size()
    return total_bytes


# Test model
test_model = BugPredictionModel(vocab_size=len(tokenizer.vocab))
print(f"Model parameters: {sum(p.numel() for p in test_model.parameters()):,}")
print(f"Model size: {calculate_model_size(test_model) / 1024:.2f} KB")

## 4. Metrics Classes

In [ ]:
@dataclass
class ClassificationMetrics:
    """Store classification metrics."""
    accuracy: float = 0.0
    precision: float = 0.0
    recall: float = 0.0
    f1_score: float = 0.0
    loss: float = 0.0
    
    def to_dict(self):
        return asdict(self)


@dataclass
class ExperimentResult:
    """Store experiment results."""
    experiment_name: str
    metrics: ClassificationMetrics
    training_time: float = 0.0
    communication_bytes: float = 0.0
    round_metrics: List[Dict] = field(default_factory=list)
    
    def to_dict(self):
        return {
            'experiment_name': self.experiment_name,
            'metrics': self.metrics.to_dict(),
            'training_time': self.training_time,
            'communication_bytes': self.communication_bytes,
            'round_metrics': self.round_metrics
        }


def calculate_metrics(y_true, y_pred, loss=0.0) -> ClassificationMetrics:
    """Calculate all classification metrics."""
    return ClassificationMetrics(
        accuracy=accuracy_score(y_true, y_pred),
        precision=precision_score(y_true, y_pred, zero_division=0),
        recall=recall_score(y_true, y_pred, zero_division=0),
        f1_score=f1_score(y_true, y_pred, zero_division=0),
        loss=loss
    )

## 5. Poisoning Attack Implementation

In [ ]:
class PoisoningAttacker:
    """
    Implements poisoning attacks for Flower.
    
    1. Model Poisoning: Add 100 to gradients/weights
    2. Data Poisoning: Flip labels (buggy <-> non-buggy)
    """
    
    GRADIENT_POISON_VALUE = 100.0
    
    @staticmethod
    def apply_model_poisoning(parameters: List[np.ndarray]) -> List[np.ndarray]:
        """
        Apply model poisoning by adding 100 to all parameters.
        Simulates malicious gradient/weight updates.
        """
        poisoned = []
        for param in parameters:
            poisoned.append(param + PoisoningAttacker.GRADIENT_POISON_VALUE)
        return poisoned
    
    @staticmethod
    def apply_data_poisoning(labels: np.ndarray) -> np.ndarray:
        """
        Apply data poisoning by flipping labels.
        buggy (1) -> non-buggy (0)
        non-buggy (0) -> buggy (1)
        """
        return 1 - labels


print("Poisoning Attack Classes Defined")
print(f"- Model Poisoning: Add {PoisoningAttacker.GRADIENT_POISON_VALUE} to parameters")
print("- Data Poisoning: Flip labels (0 <-> 1)")

## 6. Flower Client Implementation

In [ ]:
class FlowerBugClient:
    """Flower client for bug prediction with optional poisoning."""
    
    def __init__(
        self,
        client_id: int,
        model: nn.Module,
        train_data: Dict,
        is_malicious: bool = False,
        attack_type: str = 'none'
    ):
        self.client_id = client_id
        self.model = model
        self.is_malicious = is_malicious
        self.attack_type = attack_type
        self.device = next(model.parameters()).device
        
        # Prepare data
        labels = train_data['labels'].copy()
        if is_malicious and attack_type == 'data_poisoning':
            labels = PoisoningAttacker.apply_data_poisoning(labels)
            print(f"  Client {client_id}: Data poisoned (labels flipped)")
        
        self.train_loader = DataLoader(
            TensorDataset(
                torch.tensor(train_data['tokens'], dtype=torch.long),
                torch.tensor(train_data['features'], dtype=torch.float32),
                torch.tensor(labels, dtype=torch.float32)
            ),
            batch_size=32,
            shuffle=True
        )
    
    def get_parameters(self) -> List[np.ndarray]:
        return get_model_parameters(self.model)
    
    def set_parameters(self, parameters: List[np.ndarray]):
        set_model_parameters(self.model, parameters)
    
    def train(self, epochs: int = 1) -> Tuple[List[np.ndarray], int, Dict]:
        """Train model and return updated parameters."""
        self.model.train()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        criterion = nn.BCELoss()
        
        total_loss = 0.0
        num_batches = 0
        
        for _ in range(epochs):
            for tokens, features, labels in self.train_loader:
                tokens = tokens.to(self.device)
                features = features.to(self.device)
                labels = labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(tokens, features)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                num_batches += 1
        
        # Get updated parameters
        parameters = self.get_parameters()
        
        # Apply model poisoning if malicious
        if self.is_malicious and self.attack_type == 'model_poisoning':
            parameters = PoisoningAttacker.apply_model_poisoning(parameters)
        
        avg_loss = total_loss / num_batches if num_batches > 0 else 0
        
        return parameters, len(self.train_loader.dataset), {'loss': avg_loss}

## 7. Federated Learning Training Function

In [ ]:
def federated_averaging(client_updates: List[Tuple[List[np.ndarray], int]]) -> List[np.ndarray]:
    """Perform FedAvg aggregation."""
    total_samples = sum(num_samples for _, num_samples, _ in client_updates)
    
    # Initialize with zeros
    aggregated = [np.zeros_like(client_updates[0][0][i]) for i in range(len(client_updates[0][0]))]
    
    # Weighted average
    for params, num_samples, _ in client_updates:
        weight = num_samples / total_samples
        for i, param in enumerate(params):
            aggregated[i] += param * weight
    
    return aggregated


def run_flower_experiment(
    client_data_list: List[Dict],
    vocab_size: int,
    num_rounds: int = 10,
    malicious_clients: List[int] = None,
    attack_type: str = 'none',
    experiment_name: str = 'baseline'
) -> ExperimentResult:
    """
    Run Flower federated learning experiment.
    
    Args:
        client_data_list: List of client data dictionaries
        vocab_size: Vocabulary size
        num_rounds: Number of FL rounds
        malicious_clients: Indices of malicious clients
        attack_type: 'none', 'model_poisoning', or 'data_poisoning'
        experiment_name: Name for this experiment
    
    Returns:
        ExperimentResult with all metrics
    """
    if malicious_clients is None:
        malicious_clients = []
    
    print(f"\n{'='*60}")
    print(f"Running Experiment: {experiment_name}")
    print(f"Attack Type: {attack_type}")
    print(f"Malicious Clients: {malicious_clients}")
    print(f"{'='*60}")
    
    start_time = time.time()
    round_metrics = []
    
    # Create global model
    global_model = BugPredictionModel(vocab_size=vocab_size).to(device)
    model_size = calculate_model_size(global_model)
    total_communication = 0
    
    # Create clients
    clients = []
    for i, data in enumerate(client_data_list):
        is_malicious = i in malicious_clients
        client_model = BugPredictionModel(vocab_size=vocab_size).to(device)
        client = FlowerBugClient(
            client_id=i,
            model=client_model,
            train_data=data,
            is_malicious=is_malicious,
            attack_type=attack_type
        )
        clients.append(client)
    
    # Get initial global parameters
    global_params = get_model_parameters(global_model)
    
    # Training loop
    for round_num in range(1, num_rounds + 1):
        # Distribute global model to clients
        for client in clients:
            client.set_parameters(global_params)
        
        # Client training
        client_updates = []
        round_losses = []
        
        for client in clients:
            params, num_samples, metrics = client.train(epochs=1)
            client_updates.append((params, num_samples, metrics))
            round_losses.append(metrics['loss'])
        
        # Aggregate
        global_params = federated_averaging(client_updates)
        set_model_parameters(global_model, global_params)
        
        # Track communication (upload + download for all clients)
        round_comm = model_size * len(clients) * 2
        total_communication += round_comm
        
        # Quick evaluation
        global_model.eval()
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for data in client_data_list:
                tokens_t = torch.tensor(data['tokens'], dtype=torch.long).to(device)
                features_t = torch.tensor(data['features'], dtype=torch.float32).to(device)
                outputs = global_model(tokens_t, features_t)
                preds = (outputs > 0.5).cpu().numpy().astype(int)
                all_preds.extend(preds)
                all_labels.extend(data['labels'])
        
        round_acc = accuracy_score(all_labels, all_preds)
        round_loss = np.mean(round_losses)
        
        round_metrics.append({
            'round': round_num,
            'loss': round_loss,
            'accuracy': round_acc,
            'communication_bytes': round_comm
        })
        
        if round_num % 2 == 0 or round_num == num_rounds:
            print(f"  Round {round_num}/{num_rounds} - Loss: {round_loss:.4f}, Acc: {round_acc:.4f}")
    
    training_time = time.time() - start_time
    
    # Final evaluation
    print("\nFinal Evaluation...")
    global_model.eval()
    
    all_tokens = np.concatenate([d['tokens'] for d in client_data_list])
    all_features = np.concatenate([d['features'] for d in client_data_list])
    all_labels = np.concatenate([d['labels'] for d in client_data_list])
    
    with torch.no_grad():
        tokens_t = torch.tensor(all_tokens, dtype=torch.long).to(device)
        features_t = torch.tensor(all_features, dtype=torch.float32).to(device)
        labels_t = torch.tensor(all_labels, dtype=torch.float32).to(device)
        
        outputs = global_model(tokens_t, features_t)
        y_pred = (outputs > 0.5).cpu().numpy().astype(int)
        
        criterion = nn.BCELoss()
        eval_loss = criterion(outputs, labels_t).item()
    
    final_metrics = calculate_metrics(all_labels, y_pred, eval_loss)
    
    print(f"\nFinal Metrics:")
    print(f"  Accuracy:  {final_metrics.accuracy:.4f}")
    print(f"  Precision: {final_metrics.precision:.4f}")
    print(f"  Recall:    {final_metrics.recall:.4f}")
    print(f"  F1 Score:  {final_metrics.f1_score:.4f}")
    print(f"  Loss:      {final_metrics.loss:.4f}")
    print(f"  Time:      {training_time:.2f}s")
    print(f"  Comm:      {total_communication/1e6:.2f} MB")
    
    return ExperimentResult(
        experiment_name=experiment_name,
        metrics=final_metrics,
        training_time=training_time,
        communication_bytes=total_communication,
        round_metrics=round_metrics
    )

## 8. Run Experiments

In [ ]:
# Configuration
CONFIG = {
    'num_clients': 10,
    'num_rounds': 10,
    'malicious_fractions': [0.1, 0.2, 0.3]
}

# Partition data
client_data = processor.partition_iid(tokens, features, labels, CONFIG['num_clients'])
vocab_size = len(tokenizer.vocab)

print(f"Configuration:")
print(f"  Clients: {CONFIG['num_clients']}")
print(f"  Rounds: {CONFIG['num_rounds']}")
print(f"  Vocabulary size: {vocab_size}")
print(f"  Samples per client: ~{len(labels) // CONFIG['num_clients']}")

In [ ]:
# Store all results
all_results = {}

# 1. BASELINE
print("\n" + "="*70)
print("EXPERIMENT 1: BASELINE (No Attack)")
print("="*70)

baseline_result = run_flower_experiment(
    client_data_list=client_data,
    vocab_size=vocab_size,
    num_rounds=CONFIG['num_rounds'],
    malicious_clients=[],
    attack_type='none',
    experiment_name='baseline'
)
all_results['baseline'] = baseline_result

In [ ]:
# 2. MODEL POISONING
print("\n" + "="*70)
print("EXPERIMENTS: MODEL POISONING (Parameter +100)")
print("="*70)

for fraction in CONFIG['malicious_fractions']:
    num_malicious = int(CONFIG['num_clients'] * fraction)
    malicious_indices = list(range(num_malicious))
    
    exp_name = f'model_poisoning_{int(fraction*100)}pct'
    result = run_flower_experiment(
        client_data_list=client_data,
        vocab_size=vocab_size,
        num_rounds=CONFIG['num_rounds'],
        malicious_clients=malicious_indices,
        attack_type='model_poisoning',
        experiment_name=exp_name
    )
    all_results[exp_name] = result

In [ ]:
# 3. DATA POISONING
print("\n" + "="*70)
print("EXPERIMENTS: DATA POISONING (Label Flipping)")
print("="*70)

for fraction in CONFIG['malicious_fractions']:
    num_malicious = int(CONFIG['num_clients'] * fraction)
    malicious_indices = list(range(num_malicious))
    
    exp_name = f'data_poisoning_{int(fraction*100)}pct'
    result = run_flower_experiment(
        client_data_list=client_data,
        vocab_size=vocab_size,
        num_rounds=CONFIG['num_rounds'],
        malicious_clients=malicious_indices,
        attack_type='data_poisoning',
        experiment_name=exp_name
    )
    all_results[exp_name] = result

## 9. Results Summary

In [ ]:
# Print summary table
print("\n" + "="*90)
print("RESULTS SUMMARY - FLOWER BUG PREDICTION")
print("="*90)

print(f"\n{'Experiment':<30} {'Accuracy':>10} {'F1 Score':>10} {'Precision':>10} {'Recall':>10} {'Loss':>10}")
print("-"*90)

for name, result in all_results.items():
    m = result.metrics
    print(f"{name:<30} {m.accuracy:>10.4f} {m.f1_score:>10.4f} {m.precision:>10.4f} {m.recall:>10.4f} {m.loss:>10.4f}")

# Calculate drops
print("\n" + "-"*90)
print("METRIC DROPS FROM BASELINE")
print("-"*90)

baseline_m = all_results['baseline'].metrics
print(f"\n{'Experiment':<30} {'Acc Drop':>10} {'F1 Drop':>10} {'Prec Drop':>10} {'Rec Drop':>10} {'Loss Inc':>10}")
print("-"*90)

for name, result in all_results.items():
    if name == 'baseline':
        continue
    m = result.metrics
    print(f"{name:<30} {baseline_m.accuracy - m.accuracy:>+10.4f} {baseline_m.f1_score - m.f1_score:>+10.4f} {baseline_m.precision - m.precision:>+10.4f} {baseline_m.recall - m.recall:>+10.4f} {m.loss - baseline_m.loss:>+10.4f}")

## 10. Visualization

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

experiments = list(all_results.keys())
accuracies = [all_results[e].metrics.accuracy for e in experiments]
f1_scores = [all_results[e].metrics.f1_score for e in experiments]
precisions = [all_results[e].metrics.precision for e in experiments]
recalls = [all_results[e].metrics.recall for e in experiments]

x = np.arange(len(experiments))
colors = ['green' if 'baseline' in e else 'orange' if 'model' in e else 'red' for e in experiments]

for ax, data, title in zip(axes.flat, 
                           [accuracies, f1_scores, precisions, recalls],
                           ['Accuracy', 'F1 Score', 'Precision', 'Recall']):
    ax.bar(x, data, color=colors)
    ax.set_title(f'{title} by Experiment', fontsize=12)
    ax.set_ylabel(title)
    ax.set_xticks(x)
    ax.set_xticklabels([e.replace('_', '\n') for e in experiments], rotation=45, ha='right', fontsize=8)

plt.suptitle('Flower Bug Prediction: Impact of Poisoning Attacks', fontsize=14)
plt.tight_layout()
plt.savefig('flower_poisoning_results.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for name, result in all_results.items():
    rounds = [m['round'] for m in result.round_metrics]
    losses = [m['loss'] for m in result.round_metrics]
    accs = [m['accuracy'] for m in result.round_metrics]
    
    style = '-' if 'baseline' in name else '--' if 'model' in name else ':'
    axes[0].plot(rounds, losses, label=name, linestyle=style)
    axes[1].plot(rounds, accs, label=name, linestyle=style)

axes[0].set_xlabel('Round')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss by Round')
axes[0].legend(fontsize=8)
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Round')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training Accuracy by Round')
axes[1].legend(fontsize=8)
axes[1].grid(True, alpha=0.3)

plt.suptitle('Flower Training Curves: Baseline vs Poisoning Attacks', fontsize=14)
plt.tight_layout()
plt.savefig('flower_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Export Results

In [ ]:
# Prepare export
export_results = {
    'framework': 'Flower',
    'config': CONFIG,
    'experiments': {name: result.to_dict() for name, result in all_results.items()},
    'baseline_metrics': all_results['baseline'].metrics.to_dict(),
    'communication': {
        'total_bytes': all_results['baseline'].communication_bytes,
        'per_round_bytes': all_results['baseline'].communication_bytes / CONFIG['num_rounds']
    },
    'poisoning_analysis': {
        'model_poisoning': {},
        'data_poisoning': {}
    }
}

for name, result in all_results.items():
    if 'model_poisoning' in name:
        fraction = name.split('_')[-1]
        export_results['poisoning_analysis']['model_poisoning'][fraction] = {
            'metrics_after': result.metrics.to_dict(),
            'accuracy_drop': baseline_m.accuracy - result.metrics.accuracy,
            'f1_drop': baseline_m.f1_score - result.metrics.f1_score,
            'precision_drop': baseline_m.precision - result.metrics.precision,
            'recall_drop': baseline_m.recall - result.metrics.recall,
            'loss_increase': result.metrics.loss - baseline_m.loss
        }
    elif 'data_poisoning' in name:
        fraction = name.split('_')[-1]
        export_results['poisoning_analysis']['data_poisoning'][fraction] = {
            'metrics_after': result.metrics.to_dict(),
            'accuracy_drop': baseline_m.accuracy - result.metrics.accuracy,
            'f1_drop': baseline_m.f1_score - result.metrics.f1_score,
            'precision_drop': baseline_m.precision - result.metrics.precision,
            'recall_drop': baseline_m.recall - result.metrics.recall,
            'loss_increase': result.metrics.loss - baseline_m.loss
        }

with open('flower_results.json', 'w') as f:
    json.dump(export_results, f, indent=2)

print("Results saved to flower_results.json")

In [ ]:
# Download results
from google.colab import files

files.download('flower_results.json')
files.download('flower_poisoning_results.png')
files.download('flower_training_curves.png')

print("\nDownload complete! Files:")
print("  - flower_results.json")
print("  - flower_poisoning_results.png")
print("  - flower_training_curves.png")

## Summary

This notebook evaluated Flower for bug prediction with security analysis:

### Experiments Run:
1. **Baseline**: No attacks
2. **Model Poisoning** (10%, 20%, 30%): Added +100 to parameters
3. **Data Poisoning** (10%, 20%, 30%): Flipped labels

### Metrics Collected:
- Accuracy, Precision, Recall, F1 Score, Loss
- Communication cost (bytes per round)
- Training time

### Next Steps:
Run 03_Comparison_Analysis.ipynb to compare TFF and Flower results